导入数据

In [1]:
import pandas as pd
import numpy as np

train_data = pd.read_csv('data/ijcai2016_koubei_train', header=None)
train_data.columns = ['user_id', 'merchant_id', 'location_id', 'time_stamp']
print len(train_data)
print train_data.head()

merchant_info = pd.read_csv('data/ijcai2016_merchant_info', header=None)
merchant_info.columns = ['merchant_id', 'budget', 'location_id_list']
print len(merchant_info)
print merchant_info.head()

test_data = pd.read_csv('data/ijcai2016_koubei_test', header=None)
test_data.columns = ['user_id', 'location_id']
print len(test_data)
print test_data.head()

1081724
   user_id  merchant_id  location_id  time_stamp
0  1027765         4822          172    20151028
1   141398         4822          172    20151115
2   141398         4822          172    20151005
3  1027765         4822          172    20151026
4   141398         4822          172    20151011
5910
   merchant_id  budget location_id_list
0          128     331              180
1         3712     100               34
2         1409     100              317
3         3201     100              337
4         8577     100              178
473533
   user_id  location_id
0   753792          112
1  1278336          178
2  1409408          461
3   492160          214
4  1671808          481


统计train中一个location最多有几个merchant

In [2]:
location_merchant_count = train_data\
	.groupby(by=['location_id', 'merchant_id'])\
	.size()\
	.reset_index()\
	.rename(columns={0:'count'})
# print len(location_merchant_count)
print location_merchant_count.head(10)
# print location_merchant_count.columns

   location_id  merchant_id  count
0            1         1225      2
1            1         3222      3
2            1         3780      2
3            1         4417      3
4            1         7547      2
5            1         9973      3
6            2          500    210
7            2         1556   2799
8            2         2606     10
9            2         4233      2


In [3]:
location_merchant_list = location_merchant_count.groupby('location_id').apply(
	lambda s: 
	np.array2string(
		s['merchant_id'][s['count'].nlargest(20).index].values, separator=':'
	)[1:-1].replace(' ','')
).reset_index()\
	.rename(columns={0:'merchant_list'})
print location_merchant_list.head()

   location_id                                      merchant_list
0            1                      3222:4417:9973:1225:3780:7547
1            2             1556:500:8733:5805:2606:8869:8340:4233
2            4                                6344:2043:3266:9480
3            5                                                609
4            6  820:4264:171:2606:9485:8335:365:7249:5231:7834...


In [4]:
history_place=train_data[['user_id', 'location_id', 'merchant_id']]\
	.drop_duplicates()\
	.groupby(by=['user_id','location_id'])\
	.apply(
	lambda s:
	np.array2string(s['merchant_id'].values,separator=':')
	[1:-1].replace(' ',''))\
	.reset_index()\
	.rename(columns={0:'merchant_list'})
print history_place.head(20)

    user_id  location_id merchant_list
0         4          254          3527
1        22          371          4264
2        38          272           820
3        42          337           820
4        46          348          9236
5        55          438           820
6        89          188          7667
7        97           84          3538
8       108           30          8646
9       114           60          5231
10      123          308          7755
11      128           52          9485
12      129          102           210
13      136          188     7194:7312
14      159          138      221:8898
15      161           76          7682
16      165          371          5231
17      171           65           959
18      178          438           820
19      189          368          9968


In [8]:
def get_top_10(group):
	his=group['merchant_list_x']
	hot=group['merchant_list_y']
	if pd.isnull(his):
		if pd.isnull(hot):
			return 
		else:
			merchant_id_list=':'.join(hot.split(':')[0:10])
	else:
		if pd.isnull(hot):
			merchant_id_list=':'.join(his.split(':')[0:10])
		else:
			l1=(his+':'+hot).split(':')
			l2 = sorted(set(l1),key=l1.index)
			merchant_id_list=':'.join(l2[0:10])
	return merchant_id_list

history_with_hot = test_data\
	.merge(history_place, on=['user_id', 'location_id'], how='left')\
	.merge(location_merchant_list, on='location_id',how='left')
history_with_hot['merchant_id_list'] = history_with_hot.apply(get_top_10,axis=1)
print len(history_with_hot)
history_with_hot.head(10)

473533


,user_id,location_id,merchant_list_x,merchant_list_y,merchant_id_list
0,753792,112,NaN,820:5231:9565:7812:7357:5628:9164:571:2294:509...,820:5231:9565:7812:7357:5628:9164:571:2294:5095
1,1278336,178,NaN,2803:345:5777:3594:306:6252:271:5074:7696:9221...,2803:345:5777:3594:306:6252:271:5074:7696:9221
2,1409408,461,NaN,5231:820:4565:9921:281:131:601:538:3285:5553:2...,5231:820:4565:9921:281:131:601:538:3285:5553
3,492160,214,NaN,4811:414:7427:5578:6392:9328:8884:5256:9666:29...,4811:414:7427:5578:6392:9328:8884:5256:9666:2957
4,1671808,481,NaN,404:8817:4407:958:629:7392:2463:4725:9228:2100...,404:8817:4407:958:629:7392:2463:4725:9228:2100
5,427392,172,820:4796:5231,820:4264:2606:5231:1012:9793:53:446:7830:4822:...,820:4796:5231:4264:2606:1012:9793:53:446:7830
6,1279360,254,NaN,4264:5231:6691:890:625:2156:3527:508:6697:2715...,4264:5231:6691:890:625:2156:3527:508:6697:2715
7,296576,253,NaN,2886:258:6044:8817:4999:278:1075:610:487:5399:...,2886:258:6044:8817:4999:278:1075:610:487:5399
8,886400,34,NaN,8586:8403:404:4264:1108:2332:9504:57:7682:1456...,8586:8403:404:4264:1108:2332:9504:57:7682:1456
9,1934976,8,9968,9968:8817:626:621:290:317:6421:564:7328:1371:1...,9968:8817:626:621:290:317:6421:564:7328:1371


把train中存在的(user_id,location_id,merchant_id)作为预测
剩下的(user_id,location_id)取该location中hottest的top n个merchant_id

In [13]:
predict=history_with_hot.drop(['merchant_list_x','merchant_list_y'],axis=1)
predict.to_csv('data/predict.csv',header=False,index=False)
print len(predict)
predict.head()

473533


,user_id,location_id,merchant_id_list
0,753792,112,820:5231:9565:7812:7357:5628:9164:571:2294:5095
1,1278336,178,2803:345:5777:3594:306:6252:271:5074:7696:9221
2,1409408,461,5231:820:4565:9921:281:131:601:538:3285:5553
3,492160,214,4811:414:7427:5578:6392:9328:8884:5256:9666:2957
4,1671808,481,404:8817:4407:958:629:7392:2463:4725:9228:2100
